## Задание 1(4)

### Построение модели линейной регрессии

#### Обозначения: 
* $n$ - количество наблюдений
* $m$ - количество параметров (в этой модели $m = 4$)
* $X$ - матрица переменных
* $Y$ - вектор наблюдений зависимой переменной
* $S^2(b)$ - квадратичная функция ошибки модели с коэффициентами $b$
* $A = X^TX$

#### Построение модели и вычисление статистик:
1. В качестве матрицы переменных $X$ берем столбцы `song_duration_ms`, `danceability`, `energy` и добавляем еще один признак с 1 для свободной переменной (поэтому $m = 4$)

    В качестве вектора наблюдений зависимой переменной $Y$ берем столбец `song_popularity`

2. Оцениваем вектор коэффициентов с помощью ОНК по формуле:
$$\hat{b} = argmin_{b}(S^2(b)) = A^{-1} X^T Y$$

3. Формула для остаточной дисперсии (оценки дисперсии ошибки):
$$\sigma^2 = \frac{S^2(\hat{b})}{n - m} = \frac{1}{n - m} \sum_{i=1}^{n} (Y_i - \hat{Y}_i)^2$$

4. Предполагая, что $\varepsilon \sim N(0, \sigma^2I_n)$, из основной теоремы о нормальной регрессии имеем 
    * $S^2(\hat{b})$ и $\hat{b}$ независимы
    * $\hat{b} \sim N(b, \sigma^2 A^{-1})$
    * $\frac{S^2(\hat{b})}{\sigma^2} \sim \chi^2(n-m)$
    
   Тогда для коэффициента $b_i$:
   $$\sqrt{n-m} \frac{\hat{b}_i - b_i}{S(\hat{b}) \sqrt{A^{-1}_{ii}}} 
   = \frac{\frac{\hat{b}_i - b_i}{\sigma \sqrt{A^{-1}_{ii}}}}{\sqrt{\frac{1}{n-m} \frac{S^2(\hat{b})}{\sigma^2}}}
   \sim T(n - m)$$
   
   Доверительный интервал уровня $1 - \alpha$ для $b_i$:
   $$\left[ \hat{b}_i - q_{1 - \frac{\alpha}{2}} \cdot \frac{S(\hat{b}) \sqrt{A^{-1}_{ii}}}{\sqrt{n-m}} \text{; }
   \hat{b}_i - q_{\frac{\alpha}{2}} \cdot \frac{S(\hat{b}) \sqrt{A^{-1}_{ii}}}{\sqrt{n-m}} \right] \text{,}$$
   где $q_{\frac{\alpha}{2}}, \: q_{1 - \frac{\alpha}{2}}$ - квантили распределения Стьюдента $T(n-m)$

5. Из предыдущего пункта $\frac{S^2(\hat{b})}{\sigma^2} \sim \chi^2(n-m)$

   Тогда доверительный интервал уровня $1 - \alpha$ для $\sigma^2$:
   $$\left[ \frac{S^2(\hat{b})}{q_{1 - \frac{\alpha}{2}}} \text{; } 
   \frac{S^2(\hat{b})}{q_{\frac{\alpha}{2}}} \right] \text{, }$$
   где $q_{\frac{\alpha}{2}}, \: q_{1 - \frac{\alpha}{2}}$ - квантили распределения хи-квадрат $\chi^2(n-m)$

6. Формула для коэффициента детерминации:
$$R^2 = r_{Y\hat{Y}}^2 = \left( \frac{\sum_{i=1}^n (Y_i - \overline{Y})(\hat{Y}_i - \overline{\hat{Y}})}
{\sqrt{\sum_{i=1}^n (Y_i - \overline{Y})^2 \cdot \sum_{i=1}^n (\hat{Y}_i - \overline{\hat{Y}})^2}} \right)^2
= 1 - \frac{S^2(\hat{b})}{\sum_{i = 1}^n (Y_i - \overline{Y})^2}$$

#### Проведение статистических тестов
1. "Чем больше энергичность, тем больше популярность"
   
    Тест: t-тест для линейной регрессии
   
    Гипотезы:
    * $H_0$: $b_2 = 0$
    * $H_1$: $b_2 > 0$
    
    Статистика:
    $$\sqrt{n-m} \frac{\hat{b}_2 - b_2}{S(\hat{b}) \sqrt{A^{-1}_{22}}} \sim T(n-m)$$

    Тип: левосторонний
   
2. "Популярность зависит от продолжительности"
    
    Тест: t-тест для линейной регрессии
   
    Гипотезы:
    * $H_0$: $b_0 = 0$
    * $H_1$: $b_0 \neq 0$
    
    Статистика:
    $$\sqrt{n-m} \frac{\hat{b}_0 - b_0}{S(\hat{b}) \sqrt{A^{-1}_{22}}} \sim T(n-m)$$

    Тип: двусторонний
   
3. Равенство одновременно нулю коэффициентов при энергичности и "танцевальности"
    
    Тест: f-тест для линейной регрессии
   
    Гипотезы:
    * $H_0$: $Tb = 0$
    * $H_1$: $Tb \neq 0$

    где $T$ - матрица $k \times m \: (k = 2)$, в которой везде 0, кроме $t_{11} = 1$, $t_{22} = 1$
    
    Статистика:
    $$\frac{n-m}{k} \frac{S^2(\hat{b}_{T,0}) - S^2(\hat{b})}{S^2(\hat{b})} 
    = \frac{n-m}{k} \frac{(T\hat{b})^TD^{-1}(T\hat{b})}{S^2(\hat{b})} \sim F(k, n-m)$$
    где $D = TA^{-1}T^T$

    Тип: правосторонний

In [63]:
import numpy as np
import numpy.linalg as la
import pandas as pd
from scipy.stats import t, chi2, f

In [64]:
def main():
    alpha = 0.05

    data = pd.read_csv('song_data.csv')
    target = 'song_popularity'
    params = ['song_duration_ms', 'danceability', 'energy']

    n = len(data)
    m = len(params) + 1

    X_raw = data[params].to_numpy()

    X = np.pad(X_raw, ((0, 0), (0, 1)), constant_values=1)
    Y = data[target].to_numpy()
    A_inv = la.inv(X.T @ X)

    b_hat = la.inv(X.T @ X) @ X.T @ Y
    sq_loss = (X @ b_hat- Y).T @ (X @ b_hat - Y)
    print("Коэффициенты: ", b_hat.tolist()) 

    A_inv_diag = np.diagonal(A_inv)
    coef = np.sqrt(sq_loss * A_inv_diag) / np.sqrt(n - m)
    b_l = b_hat - t(n - m).ppf(1 - alpha / 2) * coef
    b_r = b_hat - t(n - m).ppf(alpha / 2) * coef
    print("Доверительные интервалы для коэффициентов:")
    for i in range(len(b_hat)):
        print(f"  {i}: [{b_l[i]}; {b_r[i]}]")
     
    err_var = sq_loss / (n - m)
    print("Остаточная дисперсия: ", err_var)
        
    err_var_l = sq_loss / chi2(n - m).ppf(1 - alpha / 2)
    err_var_r = sq_loss / chi2(n - m).ppf(alpha / 2)
    print(f"Доверительный интервал для остаточной дисперсии: [{err_var_l}; {err_var_r}]")
    
    r_sq = 1 - sq_loss / (n * Y.var())
    print("Коэффициент детерминации: ", r_sq)

    print("Проведение статистических тестов:")
    # Тест 1
    stat_1 = np.sqrt(n - m) * b_hat[2] / np.sqrt(sq_loss * A_inv_diag[2])
    pvalue_1 = t(n - m).cdf(stat_1)
    result_1 = "accept" if pvalue_1 > alpha else "reject"
    print(f"  Тест 1 | stat: {stat_1}, p-value: {pvalue_1}, result: {result_1}")
    
    # Тест 2
    stat_2 = np.sqrt(n - m) * b_hat[0] / np.sqrt(sq_loss * A_inv_diag[0])
    pvalue_2 = 2 * min(t(n - m).cdf(stat_2), 1 - t(n - m).cdf(stat_2))
    result_2 = "accept" if pvalue_2 > alpha else "reject"
    print(f"  Тест 2 | stat: {stat_2}, p-value: {pvalue_2}, result: {result_2}")
    
    # Тест 3
    T = np.array([[0, 1, 0, 0], 
                  [0, 0, 1, 0]])
    k = 2
    Tb_hat = T @ b_hat
    D_inv = la.inv(T @ A_inv @ T.T)
    stat_3 = (n - m) / k * (Tb_hat.T @ D_inv @ Tb_hat) / sq_loss
    pvalue_3 = 1 - f(k, n - m).cdf(stat_3)
    result_3 = "accept" if pvalue_3 > alpha else "reject"
    print(f"  Тест 3 | stat: {stat_3}, p-value: {pvalue_3}, result: {result_3}")

if __name__ == '__main__':
    main()
    

Коэффициенты:  [-2.8502021544996213e-06, 14.478199189010462, -0.2567102836257605, 44.60966123437598]
Доверительные интервалы для коэффициентов:
  0: [-8.10033702107137e-06; 2.3999327120721284e-06]
  1: [12.478662245548266; 16.47773613247266]
  2: [-1.7185846717509219; 1.2051641044994013]
  3: [42.64033983476662; 46.57898263398533]
Остаточная дисперсия:  474.68050520294605
Доверительный интервал для остаточной дисперсии: [465.23655458059085; 484.41633176247376]
Коэффициент детерминации:  0.010946570823343826
Проведение статистических тестов:
  Тест 1 | stat: -0.3441986918076284, p-value: 0.36535035471164734, result: accept
  Тест 2 | stat: -1.0640969795656159, p-value: 0.28729841341904644, result: accept
  Тест 3 | stat: 100.80783806592524, p-value: 1.1102230246251565e-16, result: reject
